## CCF: 企业非法集资风险预测

In [21]:
import time
import math
import warnings
import numpy as np
import pandas as pd
import lightgbm as lgb

In [22]:
print("Reading...")
base_info = pd.read_csv('dataset/train/base_info.csv')

Reading...


In [23]:
base_info.describe()

,oplocdistrict,industryco,enttype,enttypeitem,state,orgid,jobid,adbusign,townsign,regtype,...,venind,enttypeminu,midpreindcode,protype,regcap,reccap,forreccap,forregcap,congro,enttypegb
count,24865.000000,24864.000000,24865.000000,16651.000000,24865.000000,2.486500e+04,2.486500e+04,24865.000000,24865.000000,24865.000000,...,8437.000000,7270.000000,0.0,34.000000,2.467400e+04,7.084000e+03,227.000000,250.000000,249.000000,24865.000000
mean,340210.536859,7885.261141,4229.008647,1646.959942,6.164569,3.416682e+17,3.548101e+17,0.002815,0.578444,1.088236,...,2.946545,1824.567538,NaN,81.705882,5.151437e+03,4.198166e+03,292.644467,1212.583484,2805.259428,4256.337261
std,12.818573,667.047545,3892.282453,1276.284531,0.582226,1.107349e+16,3.052367e+16,0.052985,0.493818,0.417715,...,0.284933,1339.867039,NaN,37.921394,6.777086e+04,3.653798e+04,1841.310231,8113.267914,18131.954146,3873.165231
min,340000.000000,0.000000,1100.000000,1110.000000,4.000000,1.410800e+17,1.410800e+17,0.000000,0.000000,1.000000,...,1.000000,1121.000000,NaN,1.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,1110.000000
25%,340202.000000,7512.000000,1100.000000,1130.000000,6.000000,3.402000e+17,3.402000e+17,0.000000,0.000000,1.000000,...,3.000000,1151.000000,NaN,99.000000,1.500000e+01,0.000000e+00,0.000000,0.000000,0.000000,1130.000000
50%,340207.000000,8040.000000,1100.000000,1150.000000,6.000000,3.402020e+17,3.402000e+17,0.000000,1.000000,1.000000,...,3.000000,1151.000000,NaN,99.000000,8.000000e+01,0.000000e+00,0.000000,0.000000,0.000000,1152.000000
75%,340221.000000,8199.000000,9600.000000,1150.000000,6.000000,3.402070e+17,4.000000e+17,0.000000,1.000000,1.000000,...,3.000000,1153.000000,NaN,99.000000,5.000000e+02,1.000000e+02,0.000000,0.000000,0.000000,9600.000000
max,340294.000000,9700.000000,9600.000000,9600.000000,20.000000,4.000000e+17,4.000000e+17,1.000000,1.000000,4.000000,...,3.000000,4533.000000,NaN,99.000000,5.000100e+06,1.278900e+06,15428.170000,88817.920450,221453.761350,9600.000000
